# 한글 문서의 분류

In [40]:
import csv

text = []
y = []
with open("movie_data.csv", encoding="utf-8") as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        #print(row)
        if row: # 줄이 있는 것이 참인 경우에만,
            text.append(row[0]) # 영화 리뷰를 text 리스트에 추가
            y.append(row[2]) # 영화 이름을 text 리스트에 추가

In [41]:
print("Num of samples: {}".format(len(text))) # 리뷰 개수
print("Movie titiles of reviews: {}".format(set(y))) # set을 통해 제목이 한번만 출력되도록

Num of samples: 1827
Movie titiles of reviews: {'코코', '신과함께', '곤지암', '인피니티 워', '라라랜드'}


In [42]:
from sklearn.model_selection import train_test_split

# 데이터를 training set와 test set으로 나눈다.

X_train, X_test, y_train, y_test = train_test_split(text, y, random_state=0) # 비율을 지정하지 않기 때문에 75:25로 분할된다.


In [43]:
len(X_train) # 전체 데이터의 75%

1370

In [44]:
from konlpy.tag import Okt # 형태소 분석기
twitter_tag = Okt() # 초기화

In [45]:
print(twitter_tag.morphs(X_train[1])) # 두번째 리뷰에 대해서 형태소 단위로 tokenize

['혹시', '나', '하고', '봤는데', '역시', '나다', ';;', '편집', '과', '사운드', '로', '주는', '작은', '공포', '영화', "'", '푸시', "'", '에서', '인상', '깊게', '봤던걸', '여기', '서', '또', '보네', ';;']


In [46]:
twitter_tag.nouns(X_train[1])

['혹시', '역시', '편집', '사운드', '공포', '영화', '푸시', '인상', '여기', '또']

In [47]:
# tokenizer 함수로 사용
def twit_tokenizer(text):
    return twitter_tag.nouns(text)

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [49]:
tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=2) # 최소 출현 2 이상, tokenizer 정의

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

#tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=3, max_df=0.90, max_features=1000, use_idf=True, sublinear_tf=True)
tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=2) #Twitter 형태소분석기에서 명사만 추출하는 함수를 tokenizer로 이용
# twit_tokenizer 대신 twitter_tag.nouns를 직접 써도 됨
# 하나의 문서에서만 출현한 단어는 쓸모가 없으므로 제외, 즉 최소 document frequency를 2로 설정

X_train_tfidf = tfidf.fit_transform(X_train) # train data 변환 -> tfidf vector
X_test_tfidf = tfidf.transform(X_test) # test data 변환 -> tfidf vector

clf = LogisticRegression() # logistic regression 분류기 선언
clf.fit(X_train_tfidf, y_train) # 분류기 학습
print('Train score', clf.score(X_train_tfidf, y_train)) # train data 예측정확도
print('Test score', clf.score(X_test_tfidf, y_test)) # test data 예측정확도
print(X_train_tfidf.shape) # 총 1156개의 명사로 이루어짐

Train score 0.8364963503649635
Test score 0.6717724288840262
(1370, 1156)


In [54]:
X_test[:10] #test data에서 앞 10개를 출력

['졸잼 최고',
 '내용, 음악 , 연기력  무엇하나 빠지는것이 없네요 특히 음악은 계속 찾아 듣게되요^^',
 '아맥2D로 느즈막히 관람.... 히어로가 많이나오지만, 이걸 꽤나 잘 버무려놓음. 뻔한스토리의 틀을 벗어나려 노력한점은 높은점수를 줄만함.... 블럭버스터액션, 영상미는 말이필요없음...... 후속편 기대됨!',
 '후반부터 쫄렸다.',
 '진짜. 솔직히 한국 공포영화중에 이렇게 소재별로인건 정말 오랜만인듯; 지들끼리 소리지르고 정신없이 우왕자왕 심지어 무섭지도않어 효과음만크고 진짜최악임ㅉㅉ',
 '소문난 잔치에 먹을거 없음..ㅜㅜ',
 'good!',
 '아 점수를 줄 수가 없네  화면은 왜그리도 흔들어 데는지........ 재미도 없고 가볍기만하고 .... 최악의 재미없는 배멀미 영화',
 '영화 보면서 펑펑물었네요~ 부모님 사랑에 대해 다시한번 생각하게 했던 영화네요^ ^',
 '슬픈 스토리지만 삶을 돌아보게 하는 영화다. 죄를 지은자는 그 벌을 고스란히 받으리라. 사회 각종범죄자들 뉘우치길 바란다.']

In [55]:
clf.predict(X_test_tfidf)[:10]

array(['인피니티 워', '라라랜드', '인피니티 워', '곤지암', '곤지암', '인피니티 워', '인피니티 워',
       '신과함께', '코코', '신과함께'], dtype='<U6')

In [56]:
print(y_test[:10]) # test data 앞 10개의 실제 영화제목

['인피니티 워', '라라랜드', '인피니티 워', '곤지암', '곤지암', '인피니티 워', '인피니티 워', '곤지암', '신과함께', '신과함께']


#### 성능을 개선하기 위한 노력

In [58]:
# 명사 외의 다른 형태소도 많이 포함
print(twitter_tag.morphs(X_train[1]))

['혹시', '나', '하고', '봤는데', '역시', '나다', ';;', '편집', '과', '사운드', '로', '주는', '작은', '공포', '영화', "'", '푸시', "'", '에서', '인상', '깊게', '봤던걸', '여기', '서', '또', '보네', ';;']


In [61]:
tfidf = TfidfVectorizer(tokenizer=twitter_tag.morphs, min_df=2) # 명사 이외의 형태소도 사용해 봄

In [63]:
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression()
clf.fit(X_train_tfidf, y_train)
print('Train score', clf.score(X_train_tfidf, y_train))
print('Test score', clf.score(X_test_tfidf, y_test))
print(X_train_tfidf.shape)

# 명사 사용만 한 것에 비해 Train과 Test의 편차 커짐 -> Overfitting?

Train score 0.8963503649635036
Test score 0.649890590809628
(1370, 2260)


In [64]:
print(twitter_tag.pos(X_train[1], norm=True, stem=True)) #pos()는 형태소와 품사를 함께 제공

[('혹시', 'Noun'), ('나', 'Josa'), ('하다', 'Verb'), ('보다', 'Verb'), ('역시', 'Noun'), ('나다', 'Verb'), (';;', 'Punctuation'), ('편집', 'Noun'), ('과', 'Josa'), ('사운드', 'Noun'), ('로', 'Josa'), ('주다', 'Verb'), ('작다', 'Adjective'), ('공포', 'Noun'), ('영화', 'Noun'), ("'", 'Punctuation'), ('푸시', 'Noun'), ("'", 'Punctuation'), ('에서', 'Josa'), ('인상', 'Noun'), ('깊다', 'Adjective'), ('보다', 'Verb'), ('여기', 'Noun'), ('서', 'Josa'), ('또', 'Noun'), ('보다', 'Verb'), (';;', 'Punctuation')]


In [65]:
def twit_tokenizer2(text): #전체를 다 사용하는 대신, 명사, 동사, 형용사만을 사용
    target_tags = ['Noun', 'Verb', 'Adjective']
    result = []
    for word, tag in twitter_tag.pos(text, norm=True, stem=True):
        if tag in target_tags:
            result.append(word)
    return result

In [66]:
print(twit_tokenizer2(X_train[1]))

['혹시', '하다', '보다', '역시', '나다', '편집', '사운드', '주다', '작다', '공포', '영화', '푸시', '인상', '깊다', '보다', '여기', '또', '보다']


In [67]:
tfidf = TfidfVectorizer(tokenizer=twit_tokenizer2, min_df=2) #명사, 동사, 형용사를 이용하여 tfidf 생성
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression()
clf.fit(X_train_tfidf, y_train)
print('Train score', clf.score(X_train_tfidf, y_train))
print('Test score', clf.score(X_test_tfidf, y_test))
print(X_train_tfidf.shape)

# 가장 뛰어난 성능이라고 한다 -> 실제로 이런 모델을 만드려고 하면 어렵겠다

Train score 0.8715328467153285
Test score 0.6849015317286652
(1370, 1584)


In [68]:
# 모든 형태소를 사용하며 동시에 품사도 알 수 있도록 

def twit_tokenizer3(text):
    #target_tags = ['Noun', 'Verb', 'Adjective']
    result = []
    for word, tag in twitter_tag.pos(text, norm=True, stem=True):
        result.append('/'.join([word, tag])) #단어의 품사를 구분할 수 있도록 함
    return result

In [69]:
tfidf = TfidfVectorizer(tokenizer=twit_tokenizer3, min_df=2)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression()
clf.fit(X_train_tfidf, y_train)
print('Train score', clf.score(X_train_tfidf, y_train))
print('Test score', clf.score(X_test_tfidf, y_test))
print(X_train_tfidf.shape)
# Test score가 올라감

Train score 0.8927007299270073
Test score 0.6739606126914661
(1370, 2023)


In [70]:
from sklearn.linear_model import RidgeClassifier
ridge_clf = RidgeClassifier(alpha = 1)
ridge_clf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))
# Train score는 높게 올라감 .. Test는 별 차이 없다.

Train set score: 0.944
Test set score: 0.676


In [71]:
#lasso를 쓰면?
from sklearn.linear_model import LogisticRegression
import numpy as np
lasso_clf = LogisticRegression(penalty='l1', solver='liblinear')
lasso_clf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(lasso_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(lasso_clf.score(X_test_tfidf, y_test)))
print('Used features count: {}'.format(np.sum(lasso_clf.coef_ != 0)), 'out of', X_train_tfidf.shape[1])

Train set score: 0.718
Test set score: 0.643
Used features count: 240 out of 2023


In [72]:
#lsa를 쓰면?
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=239, n_iter=7, random_state=42) #압축할 component의 수 지정
svd.fit(X_train_tfidf)  
print(svd.explained_variance_ratio_)  #계산된 각 component가 설명하는 분산의 비율
print(svd.explained_variance_ratio_.sum())  #선택된 component들이 설명하는 분산의 합 -> 선택한 component의 수에 따라 달라짐
print(svd.singular_values_) 
print(svd.components_.shape)

[0.01102934 0.01405529 0.01144551 0.0110093  0.00918016 0.00876149
 0.00836851 0.00795922 0.00777003 0.00737423 0.00707199 0.00665546
 0.00659729 0.00616382 0.00588976 0.00563969 0.00546929 0.00543044
 0.00524396 0.00523751 0.0051354  0.00507564 0.00494917 0.00478661
 0.00466253 0.0046021  0.00455272 0.00452943 0.00432636 0.0043043
 0.00425346 0.00416261 0.00410043 0.00401296 0.00399825 0.00396182
 0.00392779 0.00385998 0.00373325 0.00369469 0.00365484 0.00363113
 0.00357832 0.00354544 0.00349848 0.00346928 0.00337552 0.00334918
 0.00332458 0.00330956 0.00323571 0.00321667 0.00315217 0.00314053
 0.00309441 0.00307477 0.0030649  0.00301359 0.00300256 0.00296081
 0.00293836 0.00291433 0.00288938 0.00288701 0.00286437 0.00281526
 0.00280011 0.00276482 0.00274032 0.00271676 0.00269105 0.00266254
 0.00264105 0.00262961 0.00260318 0.00258836 0.00257263 0.0025549
 0.00253862 0.00252722 0.00251281 0.00248844 0.00248194 0.00245121
 0.00244755 0.00241116 0.00239864 0.0023803  0.00236743 0.002359

In [73]:
X_train_svd = svd.transform(X_train_tfidf) # 차원 축소를 통해 가장 주요한 성분 찾아 내는 것?
X_test_svd = svd.transform(X_test_tfidf)

from sklearn.linear_model import LogisticRegression
SVD_clf = LogisticRegression()
SVD_clf.fit(X_train_svd, y_train)
print('Train set score: {:.3f}'.format(SVD_clf.score(X_train_svd, y_train)))
print('Test set score: {:.3f}'.format(SVD_clf.score(X_test_svd, y_test)))

Train set score: 0.771
Test set score: 0.654


In [74]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(tokenizer = twit_tokenizer2, min_df =2).fit(X_train)

In [75]:
X_train_cv = cv.transform(X_train) # train set을 변환
print("Train set Dimension:", X_train_cv.shape) # 몇 차원이냐
X_test_cv = cv.transform(X_test) # test set을 변환
print('Test set dimension:', X_test_cv.shape)


Train set Dimension: (1370, 1584)
Test set dimension: (457, 1584)


In [76]:
# 나이브 베이즈 분류기

from sklearn.naive_bayes import MultinomialNB
NB_clf = MultinomialNB()
NB_clf.fit(X_train_cv, y_train)
print('Train set score: {:.3f}'.format(NB_clf.score(X_train_cv, y_train)))
print('Test set score: {:.3f}'.format(NB_clf.score(X_test_cv, y_test)))

Train set score: 0.801
Test set score: 0.685
